# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks/Assignments/HW2"

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop MapReduce is based on the functional programming paradigm. The main ideas of functional programming is 1) higher order functions, where functions can take other functions as inputs. MapReduce exemplifies this in that both Map and Reduce are essentially higher order functions that take other functions to do the mapping and reducing. 2) maintaining statelessness, which means no data state is changed, which is evident in MapReduce since there is no shared mutable information that MapReduce will need in order to do parallel processing. [source: live session week2 demo]

> __b)__ Shuffling is the synchronization that happens before reducing. Multiple processes will need to join up to share intermediate results or exchange state information, such as grouping by key. It is potentially costly because this step involves copying intermediate data over the network. The number of intermdiate data is potentially more than the input data. We can mitigate the cost by reducing the intermediate data through in-memory combining and/or using combiners to aggregate the intermediate data before shuffling. [source: 'Data-Intensive Text Processing with MapReduce']

> __c)__ The input and output record format of a combiner script have to be the same because this is an optional optimization step that occurs between the mappers and reducers. It should not change the correctness of the algorithm and only perform local aggregation of intermediate data from the mapper without changing the record format.[source: 'Data-Intensive Text Processing with MapReduce']

> __d)__ I can control the level of parallelization by 1) suggesting the number of mappers to run in parallel, but Hadoop will split the file based on the size of the resulting block. 2) since each input file will be assigned to a mapper, the input data can be divided into desired number of files to control how many mappers will be used. But if the files are too big, they will be split further by HDFS. 3) specifying the number of reducers, but the number should be sufficiently large so each reducer will not crash the memory. 

> __e)__ The beginning of the multi-core era and commodity hardware prompted the creation of parallel computation frameworks like Hadoop. The semiconductor industry hit a point of diminishing returns on improving components in the single core system (faster clocks, deeper pipelines, superscalar architecture, etc) and that marked the beginning of multi-core era. As the price comes down, commodity hardware allows for the scaling out of parallel processing of large data. [source: 'Data-Intensive Text Processing with MapReduce']

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Counters are used to count events that occur during execution. The built-in counter allows us to see how many records are emitted. We can create a custom counter by writing to the standard error stream to track any events in the mapper, combiner, or reducer. It can also increment only in certain situations if we put it inside an if statement. Custom counters are helpful in determining if we have a load balancing issue. [source: week3 demo]

> __b)__ A composite key is a an intermediate key where we move part of the value into the key. This allows us to do a combination of partitioning and sorting using data beyond just the key. Custom partitioning can be done on the key while we can then specify an arbitrary number of additional sorting (such as secondary or tertiary sort) based on different values in the composite key. This way, we can have scalable sorting handled by the MapReduce framework rather than the reducer functions. [source: Data Intensive Text Processing with Map Reduce]

> __c)__ The order inversion pattern is the design where the reducer can access the result of a computation before processing the data that are needed for the computation. We can emit special key-value pairs so they appear at the top once sorting is done. This way, the reducers will have access to the special key-value pair, which is usually a required element in the computation reducers need to perform as additional data is streamed in. An example for the key-value pair is some form of aggregate total so the reducers can compute frequency. We are in essence converting a sequencing of computation to sorting through coordination within MapReduce. [source: Data Intensive Text Processing with Map Reduce]


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ A given set of files are considered in total order sort if records within each file are sorted, records across files are also sorted, and the files are also sorted. An unordered total sort is almost like a total order sort except that the files are unsorted. If records within each file are sorted, but records across files are unsorted, we have a partial sort. The total sort order allows for faster search, which is important for large datasets because such optimization will mean significant speed gain. The algorithm can be designed to take advantage of the total sort. [source: Total Order Sort Notebook]

> __b)__ The shuffle/sort phase is leveraged to implement Total Order Sort. The default shuffle behavior needs to be changed to add a custom partitioner and custom sort order because this is the only way to guarantee that records across partitions are sorted. The custom partition should be carefully designed so that we can achieve Total Order Sort once we sort data in each partition and restack the partition files. [source: Total Order Sort Notebook]

> __c)__ To achieve Total Order Sort, we need a custom partitioner (partition function or partition file) on the key that will guarantee that records across partitions will be in sorted order. This way, records within each partition (under the same key) can then be sorted independently. And since the order of the partition files are not guaranteed to be in any particular order, we must then restack the partition files in sorted order. [source: Total Order Sort Notebook]

> __d)__ The inverse hash code function needs to be used in order to generate appropriate keys so that partition key ordering is maintained. [source: Total Order Sort Notebook] 

> __e)__ This function is located in the custom partitioner, which is inside the mapper. [source: Total Order Sort Notebook] 

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [5]:
!pwd

/media/notebooks/Assignments/HW2


In [6]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [7]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [8]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

In [9]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

In [10]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2019-09-18 05:35 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in real emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output of your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected then run the provided Hadoop streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Briefly explain why or why not.


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __ e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Edit your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word "assistance" occurs 8 times in spam class and 2 times in ham class. 

> __c)__ We would not be able to add sorting parameters to sort our part a results by count because the count is the value, not the key in this case. Sorting is done on the key (the words in this case) and only secondary sort can be done on the values. The reducer will need to aggregate the total count for each word so if we want the final output to be sorted by count, it will need to be done as a postprocessing step after Reduce.

> __d)__ To print the top 10 words in each class, since the output is sorted by class and then by count, I would print the first 10 words and then look for the start of the second class and print the first 10 words. 

In [11]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [12]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [13]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	0
two	0	1


In [14]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2/eda-output': No such file or directory


In [15]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5343530159256886772.jar tmpDir=null
19/09/18 05:37:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:37:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:37:29 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 05:37:29 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 05:37:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0001
19/09/18 05:37:30 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0001
19/09/18 05:37:30 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0001/
19/09/18 05:37:30 INFO mapreduce.Job: Running job: job_1568784942760_0001
19/09/18 05:37:41 INFO mapreduce.Job: Job job_1568784942760_0001 running in uber mode : false
19/09/18 05:37:41 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [16]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [17]:
# part b - write your grep command here
!grep assistance EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [18]:
# part d/e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

rm: `/user/root/HW2/eda-sort-output': No such file or directory


In [19]:
# part d/e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.partition.keypartitioner.options=-k2,2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k3,3nr\
  -files EnronEDA/results.txt \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6452867415821230469.jar tmpDir=null
19/09/18 05:39:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:39:03 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:39:04 INFO mapred.FileInputFormat: Total input paths to process : 2
19/09/18 05:39:04 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 05:39:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0002
19/09/18 05:39:05 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0002
19/09/18 05:39:05 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0002/
19/09/18 05:39:05 INFO mapreduce.Job: Running job: job_1568784942760_0002
19/09/18 05:39:14 INFO mapreduce.Job: Job job_1568784942760_0002 running in uber mode : false
19/09/18 05:39:14 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [20]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ The number of records emitted by the mappers is the same as the number received by the reducers: 31490. In a word counting perspective, this would be the total number of words in the document.

> __c)__ The mappers emitted 31490 records and reducers received 13096. The combiner is able to significantly reduce the amount of intermediate data by aggregating the count on each file processed by mappers. 

> __d)__ If the data from the mappers cannot be aggregate to reduce intermediate data volume, the combiner will not have any work to do. In this problem, if all words are unique, the combiner will not be needed. In light of that, increasing the number of mappers may decrease the usefulness of a combiner because each file may potentially contain only unique words, and therefore, there is a higher chance of the combiner not being able to do any aggregation. 

In [21]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [22]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob546409547629075619.jar tmpDir=null
19/09/18 05:47:23 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:47:24 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 05:47:24 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 05:47:24 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 05:47:25 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0003
19/09/18 05:47:25 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0003
19/09/18 05:47:25 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0003/
19/09/18 05:47:25 INFO mapreduce.Job: Running job: job_1568784942760_0003
19/09/18 05:47:34 INFO mapreduce.Job: Job job_1568784942760_0003 running in uber mode : false
19/09/18 05:47:34 INFO mapreduce.Job:  map 0% reduce 0%
19/09/

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ To obtain the conditional probability of $c$ given $d$, $\hat{P}(c|d)$, we have to know two things: 1) $\hat{P}(c)$, which is the prior probability. This represents the overall probability of $c$.  2) $\hat{P}(t_k|c)$ for all terms in the vocabulary. $\hat{P}(t_k|c)$ is a measure of how much evidence $t_k$ contributes that $c$ is the correct class. [source: Introduction to Information Retrieval]

> __b)__ For the Enron data, $\hat{P}(c)$ refers to the prior probability of a document being spam. $\hat{P}(t_k|c)$ is the conditional probability that a particular term occuring in a spam document. We can estimate $\hat{P}(c)$ by taking calculating the percentage of documents that are labeled as spam. We can estimate $\hat{P}(t_k|c)$ by dividing the number of occurrences of a particular word in the spam class by the total word count in the spam class. We will need to pass over the data twice. In the first pass, we can partition the data into 2 separate classes (ham and spam) and sort the words. In the second pass, we can keep a running total of the total word count while we aggregate the counts for each unique word. 

> __c)__ Since there are 4 docs with 3 being class 1 and 1 being class 0, the prior probability are $\hat{P}(c)$=3/4 and $\hat{P}(\bar{c})$=1/4. And we have the following conditional probabilities. 

> For class 1, there are 8 words so the denominator is 8. The numerator is the number of occurrences of that word in class 1.

> $\hat{P}($Chinese$|c)$ = 5/8 

> $\hat{P}($Beijing$|c)$ = $\hat{P}($Shanghai$|c)$ = $\hat{P}($Macao$|c)$ = 1/8

> $\hat{P}($Tokyo$|c)$ = $\hat{P}($Japan$|c)$ = 0/8 = 0

> For class 0, there are 3 words so the denominator is 3. The numerator is the number of occurrences of that word in class 0.

> $\hat{P}($Chinese$|\bar{c})$ = $\hat{P}($Tokyo$|\bar{c})$ = $\hat{P}($Japan$|\bar{c})$ = 1/3

> $\hat{P}($Beijing$|\bar{c})$ = $\hat{P}($Shanghai$|\bar{c})$ = $\hat{P}($Macao$|\bar{c})$ = 0/3 = 0

> __d)__ Let $d_5$ = Chinese Chinese Chinese Tokyo Japan.

> $\hat{P}(c|d_5)$ $\approx$ $\hat{P}(c)$ * $\hat{P}($Chinese$|c)$ * $\hat{P}($Chinese$|c)$ * $\hat{P}($Chinese$|c)$ * $\hat{P}($Tokyo$|c)$ * $\hat{P}($Japan$|c)$ = 3/4 * (5/8)$^3$ * 0 * 0 = 0

> $\hat{P}(\bar{c}|d_5)$ $\approx$ $\hat{P}(\bar{c})$ * $\hat{P}($Chinese$|\bar{c})$ * $\hat{P}($Chinese$|\bar{c})$ * $\hat{P}($Chinese$|\bar{c})$ * $\hat{P}($Tokyo$|\bar{c})$ * $\hat{P}($Japan$|\bar{c})$ = 1/4 * (1/3)$^3$ * 1/3 * 1/3 $\approx$ 0.001

> __e)__ The unsmoothed model ran into zero probability problem so when one of the conditional probability is zero, the resulting probability becomes zero and makes it difficult to do inference. The Laplace plus 1 smoothing mitigates this zero probability problem.


In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ The keys are the unique words in the document. The values, consisting of 4 numbers, are the number of occurrences of the word in each class (class 0 and class 1), followed the conditional probabilities in class 0 and class 1. The "ClassPriors" record is different from the rest because it represents information about the whole class rather than a single word. We will need the probability numbers for inference, but the number of occurrences will be useful for debugging. 

> __b)__ We use logs because multiplying the probabilities, which are floats, can cause floating point underflow. If we had an unsmoothed model, some probabilities may be zero and the log tranformation will be undefined. 

> __c)__ We can apply Laplace smoothing to avoid running into the zero probability problem for unseen words. This would be a smoothed model. 

> __d)__ The in-memory join doesn't quite violate the functional programming principles. This is not exactly a shared state between processes because the model file only contains probabilities, which do not need updating. The in-memory join should not create a bottleneck or race condition. However, since each mapper will load the probabilities into memory, this only works well if the vocabulary is small. If the vocabulary is too big and/or there is a lot of mappers, memory may run out. 

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ We can split the documents and have multiple mappers process different parts of the documents at the same time. The mappers will still look up log probabilities from the trained model and aggregate the probabilities. The key-value structure emitted would be docID and the cumulative log probabilities of the partial document being processed. The reducers would combine all the log probabilities of a particular document (with the same docID) and then check to see which class has higher probability and produce the prediction as the last step.


Run these cells to create the example corpus and model.

In [23]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Overwriting NaiveBayes/chineseTrain.txt


In [24]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Overwriting NaiveBayes/chineseTest.txt


In [25]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Overwriting NBmodel.txt


In [26]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [27]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [28]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [29]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

rm: `/user/root/HW2/chinese-output': No such file or directory


In [30]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NBmodel.txt,NaiveBayes/classify_mapper.py \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7098974862459128983.jar tmpDir=null
19/09/18 06:17:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:17:22 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:17:23 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 06:17:23 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 06:17:23 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0004
19/09/18 06:17:24 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0004
19/09/18 06:17:24 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0004/
19/09/18 06:17:24 INFO mapreduce.Job: Running job: job_1568784942760_0004
19/09/18 06:17:32 INFO mapreduce.Job: Job job_1568784942760_0004 running in uber mode : false
19/09/18 06:17:32 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [31]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [32]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). [`HINT:` _don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications_].


__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __c)__ The Laplace smoothing requires the vocabulary count (the number of unique words).

> __d)__ For option 1, if we have the total vocabulary count, we can pass it to the reducer and not need to run a second MapReduce job. However, we would need preprocessing, or a second MapReduce job, to get the vocab count in the first place. For option 2, if the vocab count is not available, it is impossible to compute the vocab count directly using 1 MapReduce job without postprocessing. Each reducer can only compute the count of unique words in its own file. Without another step to combine the count, there would be no way to add the unique word counts from each reducer together. 

> __e)__ Given that both options require additional processing beyond a single MapReduce job, getting the vocab count ahead of time is a better option because once the vocab count is determined, there is no need to recompute that. Option 2 would require that we run MapReduce twice (or MapReduce + postprocessing step) every time we need to run or debug the job. 


In [33]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.
    
INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount
OUTPUT:
    word \t ham_totalCount \t spam_totalCount \t P(ham|doc) \t P(spam|doc)


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [34]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py 

A	chinese	0,1
A	beijing	0,1
A	chinese	0,1
A	chinese	0,1
A	chinese	0,1
A	shanghai	0,1
A	chinese	0,1
A	macao	0,1
A	tokyo	1,0
A	japan	1,0
A	chinese	1,0
A	*docTotals	1,3
A	*wordTotals	3,8


In [35]:
# part b - write a unit test for your reducer here
!echo -e 'A	*docTotals	1,3\nA	*wordTotals\t1,6\nA	chinese	0,1\nA	chinese	0,1\nA	chinese	0,1\nA	chinese	0,1\nA	shanghai	0,1\nA	tokyo	1,0' | NaiveBayes/train_reducer.py

chinese	0,4,0.0,0.6666666666666666
shanghai	0,1,0.0,0.16666666666666666
tokyo	1,0,1.0,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [36]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2 | NaiveBayes/train_reducer.py

beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
japan	1,0,0.3333333333333333,0.0
macao	0,1,0.0,0.125
shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [37]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothNB-output

rm: `/user/root/HW2/unsmoothNB-output': No such file or directory


In [38]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2\
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmoothNB-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob4323052167475125489.jar tmpDir=null
19/09/18 06:47:01 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:47:01 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:47:02 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 06:47:02 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 06:47:03 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0005
19/09/18 06:47:03 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0005
19/09/18 06:47:03 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0005/
19/09/18 06:47:03 INFO mapreduce.Job: Running job: job_1568784942760_0005
19/09/18 06:47:12 INFO mapreduce.Job: Job job_1568784942760_0005 running in uber mode : false
19/09/18 06:47:12 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [39]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmoothNB-output/part-000* > NaiveBayes/unsmoothNBResults.txt

In [40]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/unsmoothNBResults.txt

beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
japan	1,0,0.3333333333333333,0.0
macao	0,1,0.0,0.125
shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [41]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [42]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py
!echo -e 'A	*docTotals	1,3\nA	*wordTotals\t1,6\nA	chinese	0,1\nA	chinese	0,1\nA	chinese	0,1\nA	chinese	0,1\nA	shanghai	0,1\nA	tokyo	1,0' | NaiveBayes/train_reducer_smooth.py

chinese	0,4,0.14285714285714285,0.4166666666666667
shanghai	0,1,0.14285714285714285,0.16666666666666666
tokyo	1,0,0.2857142857142857,0.08333333333333333
ClassPriors	1.0,3.0,0.25,0.75


In [43]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2 | NaiveBayes/train_reducer_smooth.py

beijing	0,1,0.1111111111111111,0.14285714285714285
chinese	1,5,0.2222222222222222,0.42857142857142855
japan	1,0,0.2222222222222222,0.07142857142857142
macao	0,1,0.1111111111111111,0.14285714285714285
shanghai	0,1,0.1111111111111111,0.14285714285714285
tokyo	1,0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


In [44]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smoothNB-output

rm: `/user/root/HW2/smoothNB-output': No such file or directory


In [45]:
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2\
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smoothNB-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6408305952899647390.jar tmpDir=null
19/09/18 06:48:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:48:19 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 06:48:20 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 06:48:20 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 06:48:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0006
19/09/18 06:48:21 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0006
19/09/18 06:48:21 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0006/
19/09/18 06:48:21 INFO mapreduce.Job: Running job: job_1568784942760_0006
19/09/18 06:48:31 INFO mapreduce.Job: Job job_1568784942760_0006 running in uber mode : false
19/09/18 06:48:31 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [46]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothNB-output/part-000* > NaiveBayes/smoothNBResults.txt
!cat NaiveBayes/smoothNBResults.txt

beijing	0,1,0.1111111111111111,0.14285714285714285
chinese	1,5,0.2222222222222222,0.42857142857142855
japan	1,0,0.2222222222222222,0.07142857142857142
macao	0,1,0.1111111111111111,0.14285714285714285
shanghai	0,1,0.1111111111111111,0.14285714285714285
tokyo	1,0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance, and draw some conclusions.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. 

[`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]


* __e) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __f) short response:__ What do you notice about the 'top words' we printed in `e`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'e'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q9 Student Answers:
> __d)__ The unsmoothed version performed significantly worse. This is because this model does not handle the zero probability issue well so a lot of documents end up having zero probability, causing a lot of false negatives. This means whenever we have an unseen word, the model will not predict well.

> __e)__ model_sort_mapper adds two additional fields in the output, $maxClass$ and $maxClassP$. Since we now know the class the word belongs to and the corresponding probability, we can partition based on the class (using 2 partitions since we have 2 classes) and then sort by the probability. We can then print the top 10 words by printing the first 10 words from each of the 2 partition files. 

> __f)__ The top words are rather common words, not the words that would differentiate between ham and spam. If we increase K, we will increase the bias and decrease the variance of our model. This is because as K becomes bigger, the probabilities will become more similar. As K approaches infinity, the probabilities will become the same, therefore, the variance will decrease. As K increases, more errors are introduced to the model and therefore the bias is increased. 

__Test/Train split__

In [47]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [48]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2\
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

rm: `/user/root/HW2/enron-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7123208155521122298.jar tmpDir=null
19/09/18 07:07:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:07:41 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:07:42 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 07:07:42 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 07:07:42 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0007
19/09/18 07:07:43 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0007
19/09/18 07:07:43 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0007/
19/09/18 07:07:43 INFO mapreduce.Job: Running job: job_1568784942760_0007
19/09/18 07:07:52 INFO mapreduce.Job: Job job_1568784942760_0007 running in uber mode : false
1

In [49]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2,4,0.0001725476662928134,0.00029682398337785694


In [50]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1,22,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [51]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k2,2\
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 

# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

rm: `/user/root/HW2/smooth-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7410488173441029252.jar tmpDir=null
19/09/18 07:08:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:08:29 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:08:30 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 07:08:30 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 07:08:30 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0008
19/09/18 07:08:31 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0008
19/09/18 07:08:31 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0008/
19/09/18 07:08:31 INFO mapreduce.Job: Running job: job_1568784942760_0008
19/09/18 07:08:40 INFO mapreduce.Job: Job job_1568784942760_0008 running in uber mode : false


In [52]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2,4,0.0001858045336306206,0.00027730020520215184


In [53]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1,22,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [52]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [54]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737812	 True
d8	0	-4.394449154674438	-5.565796731681498	 True
# Documents:	4
True Positives:	2
True Negatives:	2
False Positives:	0
False Negatives:	0
Accuracy	1.0000
Precision	1.0000
Recall	1.0000
F-Score	1.0000


In [55]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmooth-model

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/Unsmoothed/NBmodel.txt,NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmooth-model \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmooth-model/part-000* > NaiveBayes/Unsmoothed/results.txt

rm: `/user/root/HW2/unsmooth-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6136906923000987123.jar tmpDir=null
19/09/18 07:09:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:09:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:09:19 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 07:09:19 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 07:09:19 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0009
19/09/18 07:09:19 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0009
19/09/18 07:09:19 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0009/
19/09/18 07:09:19 INFO mapreduce.Job: Running job: job_1568784942760_0009
19/09/18 07:09:30 INFO mapreduce.Job: Job job_1568784942760_0009 running in uber mode : fals

In [56]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/Smoothed/NBmodel.txt,NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob211340780112942520.jar tmpDir=null
19/09/18 07:10:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:10:02 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:10:04 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 07:10:04 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 07:10:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0010
19/09/18 07:10:04 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0010
19/09/18 07:10:04 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0010/
19/09/18 07:10:05 INFO mapreduce.Job: Running job: job_1568784942760_0010
19/09/18 07:10:14 INFO mapreduce.Job: Job job_1568784942760_0010 running in uber mode : false
19/09/18 07:10:14 INFO map

In [57]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5000
Precision	1.0000
Recall	0.0909
F-Score	0.1667
=========== SMOOTHED MODEL ============
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.8500
Precision	0.7857
Recall	1.0000
F-Score	0.8800


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

__`part e starts here:`__

In [58]:
# part e - write your Hadoop job here (sort smoothed model on P(word|class))

# load file into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/Smoothed/NBmodel.txt {HDFS_DIR}

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-sort-model

# hadoop job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.partition.keypartitioner.options=-k3,3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options=-k4,4nr\
  -files NaiveBayes/Smoothed/NBmodel.txt,NaiveBayes/model_sort_mapper.py \
  -mapper model_sort_mapper.py \
  -reducer /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -input {HDFS_DIR}/NBmodel.txt \
  -output {HDFS_DIR}/smooth-sort-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH} 


rm: `/user/root/HW2/smooth-sort-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5214890041511687416.jar tmpDir=null
19/09/18 07:10:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:10:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 07:10:51 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 07:10:52 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 07:10:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568784942760_0011
19/09/18 07:10:52 INFO impl.YarnClientImpl: Submitted application application_1568784942760_0011
19/09/18 07:10:52 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568784942760_0011/
19/09/18 07:10:52 INFO mapreduce.Job: Running job: job_1568784942760_0011
19/09/18 07:11:01 INFO mapreduce.Job: Job job_1568784942760_0011 running in uber mode : f

In [59]:
# part e - print top words in each class
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/smooth-sort-model/part-0000{idx} | head | column -t


===== part-00000=====

ClassPriors  47.0,33.0,0.5875,0.4125                            ham  0.5875
ect          378,0,0.023473306082001735,5.546004104043037e-05   ham  0.023473306082001735
and          258,277,0.01604112473677691,0.015417891409239643   ham  0.01604112473677691
hou          203,0,0.0126347082868822,5.546004104043037e-05     ham  0.0126347082868822
in           160,157,0.009971509971509971,0.008762686484387999  ham  0.009971509971509971
for          148,153,0.00922829183698749,0.008540846320226277   ham  0.00922829183698749
on           122,95,0.007617985878855444,0.005324163939881316   ham  0.007617985878855444
enron        116,0,0.007246376811594203,5.546004104043037e-05   ham  0.007246376811594203
i            113,106,0.007060572277963582,0.00593422439132605   ham  0.007060572277963582
will         113,69,0.007060572277963582,0.003882202872830126   ham  0.007060572277963582
cat: Unable to write to output stream.

===== part-00001=====

the   453,535,0.028118419422767

### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/1S3P9be24hLspAAPuXINUgMae1v-pUZrmDRmTQ2cwPo0/edit